# Trabajo Práctico 2: Predicción de tormenta en base de Datos Meteorológicos 

### Integrantes

- Andrioli, Facundo
- Pérez, José

### Preparación de Datos: 

1. Lea el dataset limpio que se guardó en el TP1 (punto 2 de conclusiones)

In [1]:
import pandas as pd

# Cargamos el conjunto de datos
file = 'weatherAUS_output.csv'
df = pd.read_csv(file)

2. Realice una codificación adecuada de variables categóricas si es necesario, utilizando técnicas como One-Hot 
Encoding según corresponda. 

In [2]:
# Identificamos las columnas categóricas
categorical_columns = df.select_dtypes(include=['object']).columns

# Aplicamos One-Hot Encoding a las columnas categóricas
df_encoded = pd.get_dummies(df, columns=categorical_columns, drop_first=True)

# Mostramos las primeras filas del dataset transformado
print(df_encoded.head())

   MinTemp  MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
0     12.7     15.8       0.8          1.4       7.8           35.0   
1      6.2     15.1       0.0          1.8       2.1           20.0   
2      5.3     15.9       0.0          1.4       8.0           30.0   
3     11.3     15.7       0.0          1.4       1.5           52.0   
4      7.6     11.2      16.2          4.6       1.1           46.0   

   WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  ...  WindDir3pm_NNW  \
0          13.0          15.0         75.0         52.0  ...           False   
1           2.0          11.0         81.0         56.0  ...           False   
2           6.0          13.0         71.0         46.0  ...           False   
3          15.0          22.0         62.0         62.0  ...            True   
4          17.0          13.0         83.0         88.0  ...           False   

   WindDir3pm_NW  WindDir3pm_S  WindDir3pm_SE  WindDir3pm_SSE  WindDir3pm_SSW  \
0          

3. Separe el dataset en entrenamiento y prueba (70% entrenamiento, 30% prueba).

In [3]:
from sklearn.model_selection import train_test_split

# Definimos las características (X) y el objetivo (y)
X = df_encoded.drop('RainTomorrow', axis=1)
y = df_encoded['RainTomorrow']

# Separamos el dataset en entrenamiento (70%) y prueba (30%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Datos de train y de test
print("Valores de X_train:",X_train.size)
print("Valores de y_train:",y_train.size)
print("Valores de X_test:",len(X_test))
print("Valores de y_test:",len(y_test))

Valores de X_train: 352834092
Valores de y_train: 99502
Valores de X_test: 42644
Valores de y_test: 42644


4. Normalice o estandarice los atributos numéricos para asegurar que todas las variables tengan la misma escala.

In [4]:
from sklearn.preprocessing import StandardScaler

# Identificamos las columnas numéricas
numerical_columns = X_train.select_dtypes(include=['float64', 'int64']).columns

# Creamos una instancia del StandardScaler
scaler = StandardScaler()

# Ajustamos el escalador a los datos de entrenamiento y transformamos los datos de entrenamiento
X_train[numerical_columns] = scaler.fit_transform(X_train[numerical_columns])

# Utilizamos el mismo escalador para transformar los datos de prueba
X_test[numerical_columns] = scaler.transform(X_test[numerical_columns])

# Mostramos las primeras filas del conjunto de entrenamiento transformado
print(X_train.head())


        MinTemp   MaxTemp  Rainfall  Evaporation  Sunshine  WindGustSpeed  \
5426  -0.728788 -1.024421 -0.136110    -0.529909 -0.727728      -1.583889   
75590  1.160302  0.576340 -0.277473    -0.529909 -1.276662       0.281685   
9368   0.098664  1.250345 -0.277473     0.171888  1.111201       0.558067   
73772 -0.822462 -0.813794 -0.277473    -0.529909 -1.276662       0.212590   
3398   0.286012 -0.855920 -0.230352    -0.572442 -1.249215      -1.583889   

       WindSpeed9am  WindSpeed3pm  Humidity9am  Humidity3pm  ...  \
5426      -1.572602      2.009055     1.458635     0.935639  ...   
75590      0.113636      0.794151     0.783584     1.270228  ...   
9368      -0.336027      0.794151    -2.955154    -2.123465  ...   
73772      0.563299      0.131476    -0.203027    -0.880704  ...   
3398       1.575042      1.015042     0.523950     1.557019  ...   

       WindDir3pm_NNW  WindDir3pm_NW  WindDir3pm_S  WindDir3pm_SE  \
5426            False          False         False         

### Modelado y Evaluación:

1. Determine dos métricas de evaluación que considere importante medir para este problema. Justifique su elección.

#### Accuracy (Exactitud)

- Mide la proporción de instancias correctamente clasificadas en comparación con el total de instancias. Es una métrica fácil de interpretar y adecuada cuando las clases están relativamente balanceadas.

- Te proporciona una visión general del rendimiento del modelo, ya que muestra qué porcentaje de las predicciones fueron correctas.

#### F1-score

- F1-score es la media armónica de la precisión y el recall, y es especialmente útil en casos donde las clases están desbalanceadas.

- Es una métrica robusta que toma en cuenta tanto los falsos positivos como los falsos negativos, y es útil cuando es importante minimizar ambos tipos de errores.

- El F1-score te da una mejor idea del equilibrio entre precisión y recall, especialmente si una clase es más importante que otra.

2. Implemente un modelo de base usando como predicción que determine que llueve mañana si hoy llueve, y si hoy no llueve
mañana no va a llover. 

In [5]:
from sklearn.metrics import accuracy_score, confusion_matrix

def base_model_prediction(X):
  return X['RainToday'].astype(int)

# Predecir con el modelo de base
y_pred_base = base_model_prediction(X_test) 

# Evaluar el modelo de base
accuracy_base = accuracy_score(y_test, y_pred_base)
conf_matrix_base = confusion_matrix(y_test, y_pred_base)

print(f"Exactitud del modelo base: {accuracy_base}")
print("Matriz de confusión:")
print(conf_matrix_base)

Exactitud del modelo base: 0.7605759309633243
Matriz de confusión:
[[27997  5141]
 [ 5069  4437]]


3. Implemente y entrene un modelo de k Nearest Neighbors (kNN) para predecir si va a llover mañana. Utilice la 
distancia euclidiana como medida de distancia entre vecinos.Ajuste el parámetro k utilizando técnicas como la 
validación cruzada (5 folds) para optimizar el rendimiento del modelo (utilice como función de optimización una de las 
métricas definidas en el punto anterior). 

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import classification_report, accuracy_score

# Creamos un imputador que reemplace los NaN por la media de cada columna
imputer = SimpleImputer(strategy='mean')

# Aplicamos el imputador a los datos de entrenamiento y prueba
X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed = imputer.transform(X_test)

print("Definiendo el clasificador kNN...")
knn = KNeighborsClassifier(metric='euclidean')

# Definimos los parámetros para GridSearchCV
param_grid = {'n_neighbors': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]}  # Probar valores de k impares
print("Iniciando GridSearchCV para encontrar el mejor valor de k...")

# Usamos GridSearchCV para encontrar el mejor k usando validación cruzada (5 folds)
grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy', verbose=3)
grid_search.fit(X_train_imputed, y_train)

# Mostramos el mejor parámetro k y el mejor puntaje
best_k = grid_search.best_params_['n_neighbors']
best_score = grid_search.best_score_
print(f"\nMejor valor de k encontrado: {best_k}")
print(f"Puntaje de validación cruzada (accuracy) con el mejor k: {best_score:.2f}")

# Entrenamos el modelo kNN con el mejor valor de k en el conjunto de entrenamiento
print("Entrenando el modelo kNN con el mejor valor de k...")
best_knn = KNeighborsClassifier(n_neighbors=best_k, metric='euclidean')
best_knn.fit(X_train_imputed, y_train)
print("Modelo entrenado.")

# Hacemos predicciones en el conjunto de prueba
print("Haciendo predicciones en el conjunto de prueba...")
y_pred = best_knn.predict(X_test_imputed)

# Evaluar el modelo
class_report_knn = classification_report(y_test, y_pred)
accuracy_knn = accuracy_score(y_test, y_pred)
conf_matrix_knn = confusion_matrix(y_test, y_pred)

# Reporte de clasificación
print("Reporte de Clasificación:")
print(class_report_knn)

# Mostramos los resultados
print(f"Exactitud del modelo: {accuracy_knn}")
print("Matriz de confusión del modelo:")
print(conf_matrix_knn)


Definiendo el clasificador kNN...
Iniciando GridSearchCV para encontrar el mejor valor de k...
Fitting 5 folds for each of 11 candidates, totalling 55 fits
[CV 1/5] END .....................n_neighbors=1;, score=0.804 total time= 4.0min
[CV 2/5] END .....................n_neighbors=1;, score=0.806 total time= 4.1min
[CV 3/5] END .....................n_neighbors=1;, score=0.802 total time= 4.2min
[CV 4/5] END .....................n_neighbors=1;, score=0.801 total time= 4.1min
[CV 5/5] END .....................n_neighbors=1;, score=0.802 total time= 4.1min
[CV 1/5] END .....................n_neighbors=3;, score=0.826 total time= 4.2min
[CV 2/5] END .....................n_neighbors=3;, score=0.832 total time= 4.2min
[CV 3/5] END .....................n_neighbors=3;, score=0.829 total time= 4.1min
[CV 4/5] END .....................n_neighbors=3;, score=0.828 total time= 4.2min
[CV 5/5] END .....................n_neighbors=3;, score=0.826 total time= 4.2min
[CV 1/5] END .....................

4. Implemente y entrene un modelo de regresión logística para predecir la misma variable objetivo.

In [11]:
from sklearn.linear_model import LogisticRegression

# Creamos una instancia del modelo de regresión logística
log_reg = LogisticRegression(max_iter=1000, random_state=42)

# Entrenamos el modelo con los datos de entrenamiento imputados
log_reg.fit(X_train_imputed, y_train)

# Predecir en el conjunto de prueba
y_pred_log_reg = log_reg.predict(X_test_imputed)

# Evaluar el modelo
accuracy_log_reg = accuracy_score(y_test, y_pred_log_reg)
conf_matrix_log_reg = confusion_matrix(y_test, y_pred_log_reg)

# Mostramos los resultados
print(f"Exactitud del modelo de regresión logística: {accuracy_log_reg}")
print("Matriz de confusión del modelo de regresión logística:")
print(conf_matrix_log_reg)

Exactitud del modelo de regresión logística: 0.8505534190038458
Matriz de confusión del modelo de regresión logística:
[[31175  1963]
 [ 4410  5096]]


5. Evalúe los dos modelos con las métricas definidas al principio. Discuta las fortalezas y debilidades de cada enfoque 
en el contexto específico de este problema comparándolos con el modelo de base.

In [12]:
from sklearn.metrics import f1_score

# Modelo Base
f1score_base = f1_score(y_test, y_pred_base, average='macro')

print("\n--Modelo Base--")
print(f"Accuracy: {accuracy_base}")
print(f"F1-score: {f1score_base}")

# Modelo kNN
f1score_knn = f1_score(y_test, y_pred, average='macro')

print("\n--Modelo kNN--")
print(f"Accuracy: {accuracy_knn}")
print(f"F1-score: {f1score_knn}")

# Modelo Reg. Log.
f1score_log_reg = f1_score(y_test, y_pred_log_reg, average='macro')

print("\n--Modelo Regresión Log.--")
print(f"Accuracy: {accuracy_log_reg}")
print(f"F1-score: {f1score_log_reg}")


--Modelo Base--
Accuracy: 0.7605759309633243
F1-score: 0.6553882836003643

--Modelo kNN--
Accuracy: 0.846004127192571
F1-score: 0.7368743491620866

--Modelo Regresión Log.--
Accuracy: 0.8505534190038458
F1-score: 0.761269282544655


### Resumen de Resultados

| Modelo                   | Accuracy       | F1-Score          |
|--------------------------|----------------|-------------------|
| **Modelo Base**          | 0.7606         | 0.6554            |
| **k-Nearest Neighbors**  | 0.8460         | 0.7369            |
| **Regresión Logística**  | 0.8506         | 0.7613            |

### Comparación y Discusión

1. **k-Nearest Neighbors (kNN)**:
   - **Fortalezas**:
     - Mejora significativa en la **exactitud** (0.8460) y el **F1-score** (0.7369) en comparación con el modelo base.
     - Buen rendimiento en la predicción de la clase mayoritaria (clase 0).
   - **Debilidades**:
     - El **recall** para la clase minoritaria (clase 1) es bajo, lo que indica que el modelo tiene problemas para identificar correctamente los casos de esta clase.
     - Sensible a la elección de k y al balance de las clases.

2. **Regresión Logística**:
   - **Fortalezas**:
     - Presenta la mayor **exactitud** (0.8506) y **F1-score** (0.7613) entre los modelos evaluados.
     - Mejor equilibrio entre precisión y recall para ambas clases, lo que sugiere que maneja mejor el desbalance de clases.
   - **Debilidades**:
     - Aunque es el modelo con mejor rendimiento, aún presenta errores en la predicción de la clase minoritaria (clase 1), como se refleja en la matriz de confusión.


### Conclusiones:

1. Resuma los hallazgos más relevantes obtenidos.

- **Comparación con el Modelo Base**: Ambos modelos avanzados (kNN y Regresión Logística) muestran una mejora significativa en las métricas de exactitud y F1-score en comparación con el modelo base. Esto indica que ambos enfoques ofrecen un valor añadido sustancial para el problema en cuestión.
- **Comparación entre kNN y Regresión Logística**: La Regresión Logística supera ligeramente al kNN en ambas métricas, lo que sugiere que es el modelo más robusto en este contexto. Además, maneja mejor el desbalance de clases, lo que es crucial para la correcta clasificación de los datos minoritarios.

En resumen, aunque ambos modelos avanzados mejoran el desempeño respecto al modelo base, la Regresión Logística es la opción más fuerte en este contexto, debido a su mejor rendimiento general y balance entre precisión y recall.

2. Guarde los conjuntos de entrenamiento y evaluación para el siguiente TP en archivos `csv`.

In [13]:
X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_train.to_csv('y_train.csv', index=False)
y_test.to_csv('y_test.csv', index=False)

Hola profesor, estos archivos no estan el github porque son muy pesados para subirlos, por lo que le enviamos tambien el zip del tp2 donde incluye estos.

3. Guarde en formato pickle el mejor modelo, seleccionado según su criterio.

In [14]:
import pickle

best_model = log_reg  

# Guardamos el modelo regresión logística en formato pickle
with open('best_model.pkl', 'wb') as f:
    pickle.dump(best_model, f)